# Numpy : broadcasting\`

# Centering and scaling a data matrix

In the sequel $X$ is a (data) numerical matrix, that is an element of
$\mathbb{R}^{n \times p}$. The rows of $X$ (the individuals) are the
sample points. Each sample point is a tuple of $p$ elements (the
so-called variables).

The *sample mean* is defined as $$
\overline{X}= \begin{pmatrix} \frac{1}{n}\sum_{i=1}^n X_{i,j}\end{pmatrix}_{j\leq p}
$$ In linear algebra, $\overline{X}$ is the result of vector matrix
multiplication: $$
\overline{X} = \frac{1}{n}\begin{pmatrix} 1 & \ldots & 1\end{pmatrix} \times X
$$ Here, we view $\overline{X}_n$ as a row vector built from column
averages.

Centering $X$ consists in subtracting the column average from each
matrix element.
$$X - \begin{pmatrix}1 \\ \vdots \\ 1\end{pmatrix} \times \overline{X}$$

Note that centering consists on projecting the columns of $X$ on the
$n-1$ dimensional subspace of $\mathbb{R}^n$ that is orthogonal to
$\begin{pmatrix} 1 & \ldots  &  1\end{pmatrix}^\top$: Let us call $Y$
the matrix obtained from centering the columns of $X$.

Scaling $Y$ consists of dividing each coefficient of $Y$ by $1/\sqrt{n}$
times the (Euclidean) norm of its column.

Let us call $\sigma_j$ the Euclidean norm of column $j$ of $Y$
($1\leq j \leq p$) divided by $1/\sqrt{n}$: $$
\sigma_j^2 = \frac{1}{n}\sum_{i=1}^n Y_{i,j}^2 = \frac{1}{n} \sum_{i=1}^n \left(X_{i,j} - \overline{X}_j\right)^2
$$ This is also the standard deviation of the $j^{\text{n}}$ column of
$X$.

The standardized matrix $Z$ is obtained from the next multiplication $$
Z = Y \times 
  \begin{pmatrix} 
    \sigma_1 & 0        &  \ldots      &     & 0 \\
    0        & \sigma_2 &   0    &     & \vdots \\
    \vdots   & 0         & \ddots &     & \vdots  \\
    0        & \ldots   &        &     & \sigma_d       
  \end{pmatrix}^{-1}
$$

Note that for $i\leq n$, $j \leq p$:
$$Z_{i,j} = \frac{X_{i,j} - \overline{X}_j}{\sigma_j}$$

$Z$ is called the $z$-score matrix. It shows up in many statistical
analyses.

In the statistical computing environment `R`, a function called
`scale()` computes the $z$-score matrix. On may ask whether NumPy offers
such a function.

# Scaling in NumPy (standardization)

In NumPy, there is no single function equivalent to R’s `scale()`
function. However, you can achieve the same result using *broadcasting*.

In [1]:
import numpy as np

Let us first generate a toy data matrix with random (Gaussian)
coefficients. This is an opportunity to introduce `np.random`.

We will work with $n=5$ and $p=3$.

We first build a (positive definite) covariance matrix. We ensure
positive definiteness starting from the Cholesky factorization.

In [2]:
# %%
L = np.array([  
  [1., 0., 0.], 
  [.5, 1., 0.], 
  [.5, .5, 1.]])
C = L @ L.transpose()
# L is the Cholesky factor of C

In [3]:
# Generate a sample of 5 independent normal vectors with mean (1, 2, 3) and covariance C
mu = np.array([1, 2, 3])

In [4]:
X = np.random.randn(5,3) @ L.transpose() + mu

In NumPy, function `mean` with well chosen optional `axis` argument
returns a 1D array filled with column averages

> **Note**
>
> We just did something strange: we added a matrix with shape $(5,3)$
> and a vector with length $3$. In linear algebra, this is not
> legitimate. We just used the device called *broadcasting*. See below.

In [5]:
# Compute column averages
# That is compute arithmetic mean along axis `0`
emp_mean = np.mean(X, axis=0)

We can magically center the columns using broadcasting.

In [6]:
X_centered = X - emp_mean

If broadcasting were note possible, we could still achieve the result by
resorting to NumPy implementation of matrix multiplication

In [7]:
X - np.ones((5,1), dtype=np.float16) @ emp_mean.reshape(1,3)

is a centered version of `X`:

In [8]:
X_centered - (X - np.ones((5,1), dtype=np.float16) @ emp_mean.reshape(1,3))

We compute now the column standard deviations using function `np.std`
with `axis` argument set to `0`

In [9]:
emp_std = np.std(X, axis=0)

The $z$-score matrix is obtained using another broadcasting operation.

In [10]:
Z = (X - emp_mean) / emp_std  # X_centered / emp_std

Finally, let us perform the sanity checks :

In [11]:
# %%
np.mean(Z, axis=0)  # Z is column centered

In [12]:
np.std(Z, axis=0)   # Z is standardized

> **Note**
>
> Alternatively, we can use `scipy.stats.zscore` which provides R’s
> `scale()`-like functionality:
>
> ``` python
> from scipy.stats import zscore
> X_scaled_scipy = zscore(X, axis=0)
> X_scaled_scipy
> ```
>
> `scipy.stats.zscore` centers and scales the data by default
> (equivalent to R’s `scale()` with default arguments).
>
> Centering and standardization are classical preprocessing steps before
> Principal Component Analysis (and before many Machine Learning
> procedures).

# How does broadcasting work ?

## Why broadcasting ? (from the documentation)

> The term broadcasting describes how NumPy treats arrays with different
> shapes during arithmetic operations. Subject to certain constraints,
> the smaller array is “broadcast” across the larger array so that they
> have compatible shapes. Broadcasting provides a means of vectorizing
> array operations so that looping occurs in C instead of Python. It
> does this without making needless copies of data and usually leads to
> efficient algorithm implementations. There are, however, cases where
> broadcasting is a bad idea because it leads to inefficient use of
> memory that slows computation.

## How ?

> When operating on two arrays, NumPy compares their shapes
> element-wise. It starts with the trailing (i.e. rightmost) dimension
> and works its way left. Two dimensions are compatible when
>
> -   they are equal, or
> -   one of them is 1.

In our setting the shape of `X` and `emp_mean` are `(5,3)` and `(3)`.
The rightmost dimensions are equal, hence compatible.

> Input arrays do not need to have the same number of dimensions. The
> resulting array will have the same number of dimensions as the input
> array with the greatest number of dimensions, where the size of each
> dimension is the largest size of the corresponding dimension among the
> input arrays. Note that missing dimensions are assumed to have size
> one.

In our setting, `emp_mean` is (virtually) reshaped to `(1,3)` and the
two arrays are fully compatible. To match the leading dimension of `X`,
we can stack three copies of reshaped `emp_mean`, this is just like
multiplying `emp_mean` by `np.array([1.], shape=(3,1))`.

> When either of the dimensions compared is one, the other is used. In
> other words, dimensions with size 1 are stretched or “copied” to match
> the other.

# References

[Official Numpy
documentation](https://numpy.org/devdocs/user/basics.broadcasting.html#basics-broadcasting)